In [ ]:
%matplotlib inline

In [ ]:
import os
import imp
import pickle
import bcolz as bc
import numpy as np

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator as IDG

from keras.layers import Input, Dense, Flatten, Conv2D, BatchNormalization, MaxPooling2D, ELU, Dropout, Add, GlobalMaxPooling2D, Activation
from keras.applications.xception import Xception
from keras.models import Model, load_model
from keras import backend as K


In [ ]:
from config import config,bcolz_array_iterator
imp.reload(config)
imp.reload(bcolz_array_iterator)
origin_dir = config.origin_dir
group_dir = config.group_dir
bcolz_dir = config.bcolz_dir
config_dir = config.config_dir

BcolzArrayIterator = bcolz_array_iterator.BcolzArrayIterator

In [ ]:
data = bc.carray(rootdir=os.path.join(bcolz_dir,"data"),mode="r")
labels = bc.carray(rootdir=os.path.join(bcolz_dir,"labels"),mode="r")
data_valid = bc.carray(rootdir=os.path.join(bcolz_dir,"data_valid"),mode="r")
labels_valid = bc.carray(rootdir=os.path.join(bcolz_dir,"labels_valid"),mode="r")

In [ ]:
with open(os.path.join(bcolz_dir,"label_dict.pickle"),"rb") as f:
    label_dict = pickle.load(f)

from matplotlib import pyplot as plt

a=data[12458][:]
a=np.array(a,dtype=np.uint8)
plt.figure(figsize=(10,10))
plt.imshow(a)

# buildmodel

In [ ]:
def conv_block(inx,iter_num):
    x = BatchNormalization()(inx)
    x = Conv2D(96,(3,3),padding="same")(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.2)(x)
    x = ELU()(x)
    
    return x
    

In [ ]:
model_path = os.path.join(config_dir,"model.h5")
try:
    model = load_model(model_path)
except:
    input_layer = Input(data.shape[1:])
    #x = Xception(input_shape=(256,256,3),include_top=False,weights=None,pooling="avg")(input_layer)
    first_layer = Conv2D(20,(9,9),padding="same")(input_layer)  #split channel

    x = first_layer
    for i in range(2):
        x = conv_block(x,i)

    x = Conv2D(len(label_dict),(3,3),padding="same")(x)
    x = GlobalMaxPooling2D()(x)
    
    output_layer = Activation("softmax")(x)

    model = Model(input_layer,outputs=output_layer)

    model.compile("adam","sparse_categorical_crossentropy",metrics=["acc"])

In [ ]:
model.summary()

In [ ]:
batch_size = 8

In [ ]:
batch_train = BcolzArrayIterator(data,labels,batch_size=batch_size)
batch_valid = BcolzArrayIterator(data_valid,labels_valid,batch_size=batch_size)

In [ ]:
model.fit_generator(batch_train,(len(data)/batch_size),validation_data=batch_valid,validation_steps=len(data_valid),epochs=60)

In [ ]:
model.save(model_path)

In [ ]:
test_origin = data_valid[110]
test = np.expand_dims(test_origin,-1)
test = np.expand_dims(test,0)
rslt = model.predict(test)
indexes = np.argsort(rslt)[0,-10:]
prob = rslt[0][indexes]
name = [label_dict[index] for index in indexes]
from matplotlib import pyplot as plt
plt.figure(figsize=(10,10))
plt.imshow(test_origin,"gray")
name,prob

In [ ]:
np.argsort(rslt)